In [1]:
from dataclasses import dataclass, field
from typing import Optional
import sys
sys.path.append('/scratch/2023-fall-sp-le/langseg')

import torch
from torch import nn
from torch.nn import functional as F
torch.backends.cuda.matmul.allow_tf32 = True
from transformers import CLIPProcessor, CLIPModel
from diffusers import StableDiffusionPipeline

import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image
from  matplotlib import pyplot as plt
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.transforms import InterpolationMode
from methods.prompt_engineering import extract_class_embeddings, extract_clip_text_embeddings
BICUBIC = InterpolationMode.BICUBIC

from datasets.coco_stuff import coco_stuff_categories
from datasets.cityscapes import cat_to_label_id as cityscapes_cats
from datasets.utils import get_dataset
from utils.metrics import RunningScore
from utils.plotting import *

from methods.diffusion_patch import *
from methods.diffusion import *
from methods.diffusion_seg import *
from methods.diffusion_utils import *
from methods.text_embeddings import *
from methods.multilabel_classifiers import *
from methods.exp_utils import *

%load_ext autoreload
%autoreload 2

In [2]:
@dataclass
class SegmentationConfig:
    dir_dataset: str = field(
        default="/sinergia/ozaydin/segment/STEGO-master/data/cocostuff", metadata={"help": "dir dataset"}
    )
    dataset_name: str = field(
        default="voc2012", metadata={"help": "for get_dataset"}
    )
    split: str = field(
        default="train", metadata={"help": "which split to use"}
    )
    resolution: int = field(
        default=512, metadata={"help": "resolution of the images, e.g, 512, 768, 1024"}
    )
    mask_res: int = field(
        default=320, metadata={"help": "resolution of the masks, e.g, 64, 320, 512"}
    )
    dense_clip_arch: str = field(
        default="RN50x16", metadata={"help": "not used in cocostuff"}
    )

args = SegmentationConfig()

In [3]:
dataset, categories, palette = get_dataset(
    dir_dataset=args.dir_dataset,
    dataset_name=args.dataset_name,
    split=args.split,
    resolution=args.resolution,
    mask_res=args.mask_res,
    dense_clip_arch=args.dense_clip_arch
)

if args.dataset_name == "coco_stuff":
    label_id_to_cat = categories if not dataset.use_27 else coco_stuff_categories
    cat_to_label_id = {v: i for i, v in enumerate(label_id_to_cat)}
elif args.dataset_name == "cityscapes":
    cat_to_label_id = cityscapes_cats
    label_id_to_cat = {i: c for c, i in cat_to_label_id.items()}
elif args.dataset_name == "voc2012" or args.dataset_name == "pascal_context":
    label_id_to_cat = categories
    cat_to_label_id = {v: i for i, v in enumerate(label_id_to_cat)}

In [4]:
idxx = 58
val_img = dataset[idxx]["img"].cpu().numpy()
val_gt = dataset[idxx]["gt"].cpu().numpy()
lab_ids = sorted(list(np.unique(val_gt)))
lab_ids = np.array(lab_ids[1:])
val_pil_img = render_results(val_img, val_gt, palette)
_ = get_legends(lab_ids, palette, label_id_to_cat, is_voc2012 = args.dataset_name == "voc2012")

val_gt = dataset[idxx]["gt"].unsqueeze(0)
val_labels = sorted(np.unique(val_gt))
val_labels = [l for l in val_labels if l != -1] # don't process label -1 (ignored unlabelled pixels)
val_labels = [label_id_to_cat[c] for c in val_labels]
val_labels = [l for l in val_labels if l != "background"] # don't feed "background" as text input

In [5]:
import torch
from methods import gem
import requests
from PIL import Image

# model_name = 'ViT-B/16'  # 'ViT-B-16-quickgelu'
# pretrained = 'openai'  # 'metaclip_400m'
model_name = 'ViT-B/16-quickgelu'
# model_name = 'ViT-L/14-quickgelu'
pretrained = 'metaclip_400m'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# init model and image transform
gem_model = gem.create_gem_model(model_name=model_name,
                                 pretrained=pretrained, 
                                 device=device)

In [6]:
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16,
    use_safetensors=True, variant="fp16",
    # device_map="auto"
)
pipe = pipe.to("cuda")
configure_ldm(pipe)

handles = {}
attention_layers_to_use = ATTENTION_LAYERS
attention_store = AttentionStore(low_resource=False, no_uncond=True, layer_keys=attention_layers_to_use)
attention_store.num_att_layers = len(attention_layers_to_use)
handles = register_attention_hooks(pipe.unet, attention_store, attention_layers_to_use, handles)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [7]:
# def get_image_and_labels(idx):
#     img_path = dataset[idx]["p_img"]
#     image = Image.open(img_path).convert('RGB').resize((1024, 1024))
#     val_gt = dataset[idx]["gt"].cpu().numpy()
#     val_labels = sorted(np.unique(val_gt))
#     val_labels = np.array([l for l in val_labels if l > 0])
#     y_true = np.zeros((len(label_id_to_cat),), dtype=np.int64)
#     y_true[val_labels] = 1
#     return image, y_true


# def get_pred_label_names(y_pred, candidate_labels):
#     label_preds = np.array(candidate_labels)[y_pred.astype(bool)]
#     assert len(label_preds) > 0
#     return label_preds.tolist()


# # clip_model_id = "openai/clip-vit-large-patch14"
# clip_model_id = "facebook/metaclip-h14-fullcc2.5b"
# clip_classifier = CLIPMultilabelClassifier(clip_model_id, label_id_to_cat)

# # del clip_classifier.clip_model.text_model
# # clip_classifier.clip_model.to(pipe.text_encoder.dtype)
# # clip_classifier.clip_model.text_model = pipe.text_encoder.text_model

# clip_classifier.init_text_embeds()
# del clip_classifier.clip_model.text_model

In [8]:
# # load image and text
# preprocess = gem.get_gem_img_transform(img_size=(448, 448))
# image = preprocess(Image.open(dataset[idxx]["p_img"])).unsqueeze(0).to(device)
# class_names = label_id_to_cat#[1:]
# with torch.no_grad():
#     logits = gem_model(image, class_names, normalize=False, return_ori=False)  # [1, num_class, W, H]

In [9]:
# normed_logits = gem_model.min_max(logits)
# gem.visualize(image, class_names, normed_logits)  # (optional visualization)

In [10]:
def run_gem_voc():
    preprocess = gem.get_gem_img_transform(img_size=(448, 448))
    pbar = tqdm(range(len(dataset)))

    rscores = [RunningScore(len(label_id_to_cat)) for i in range(7)]

    cls_gt = []
    cls_pred_0 = []
    cls_pred_2 = []

    # 0: CA @ SA
    # 1: CA @ SA & GT class names
    # 2: CA only
    # 3: CA only & GT class names
    # 4: labeling GT clusters with cluster-avg CA
    # 5: labeling diffseg clusters with cluster-avg CA
    # 6: labeling specclust clusters with cluster-avg CA
    # for ALL cases: GT class names are NOT used to compute bg probs
    # as GT class names are only used to SELECT which classes to argmax over
    for idx in pbar:
        val_img = dataset[idx]["img"].permute(1,2,0)[None,...].numpy()
        val_gt = dataset[idx]["gt"].unsqueeze(0)
        val_labels = sorted(np.unique(val_gt))
        val_labels = [l for l in val_labels if l != -1] # don't process label -1 (ignored unlabelled pixels)
        val_labels = [label_id_to_cat[c] for c in val_labels]
        val_labels = [l for l in val_labels if l != "background"] # don't feed "background" as text input

        # # multilabel classification (does not work)
        # image, y_true = get_image_and_labels(idx)
        # y_pred, _ = clip_classifier.forward_v2(image, choice=(8,8), clf_thresh=0.25)
        # y_pred = y_pred.cpu().numpy()
        # val_labels = get_pred_label_names(y_pred, label_id_to_cat)

        # load image and text
        image = preprocess(Image.open(dataset[idx]["p_img"]).convert("RGB")).unsqueeze(0).to(device)
        class_names = label_id_to_cat[1:]

        with torch.no_grad():
            logits01 = gem_model(image, class_names, output_size=64, normalize=False)  # [1, num_class, W, H]
        # CA only
        logits23456 = F.interpolate(logits01, size=args.mask_res, mode='bilinear')
        logits23456 = get_voc2012_probs(logits23456)
        pred2 = logits23456.argmax(dim=1)
        pred3 = filter_prompt(logits23456, ["background"]+val_labels, cat_to_label_id).argmax(dim=1)
        pred4 = label_clusters(val_gt, logits23456)

        # CA @ SA
        val_img = dataset[idx]["img"].permute(1,2,0)[None,...].numpy()
        attention_store.reset()
        text_embeds = get_text_embeddings_simple(pipe.tokenizer, pipe.text_encoder, "")
        val_img = val_img.repeat(len(text_embeds), axis=0)
        training_step(pipe, text_embeds, val_img, attention_store, no_uncond=True, normalize=False, low_resource=False, timestep=100)
        ca, sa = get_attention_maps(
            attention_store.get_average_attention(),
            batch_size=len(text_embeds),
            label_indices=None,
            output_size=64,
            average_layers=True,
            apply_softmax=True,
            softmax_dim=-1,
            simple_average=False
        )
        pred5 = diffseg(sa, out_res=args.mask_res, refine=True)
        pred5 = label_clusters(pred5, logits23456)
        pred6 = get_specclust_mask(
            logits23456.permute(0, 2, 3, 1).view(1, -1, logits23456.shape[1]), sa, cat_to_label_id,
            list(range(len(label_id_to_cat))), label_id_to_cat,
            output_size=args.mask_res, minmax_norm=False
        ).long()

        ca0 = get_voc2012_probs(logits01)
        ca0 = ca0.view(*ca0.shape[:2], -1).permute(0, 2, 1)
        agg_map0 = get_agg_map(ca0, sa, minmax_norm=False, ca_norm=False)
        agg_map0 = upscale_attn(agg_map0, args.mask_res)
        pred0 = agg_map0.argmax(dim=-1)

        ca1 = get_voc2012_probs(logits01)
        ca1 = ca1.view(*ca1.shape[:2], -1).permute(0, 2, 1)
        agg_map1 = get_agg_map(ca1, sa, minmax_norm=False, ca_norm=False)
        N, C, H, W = logits01.shape
        agg_map1 = filter_prompt(
            agg_map1.permute(0, 2, 1).view(N, C+1, H, W), ["background"]+val_labels, cat_to_label_id
        ).permute(0, 2, 3, 1).view_as(agg_map1)
        agg_map1 = upscale_attn(agg_map1, args.mask_res)
        pred1 = agg_map1.argmax(dim=-1)
        # sa1, bg_mask1 = run_diffusion(val_labels, val_img)
        # ca1 = (filter_prompt(logits01, val_labels, cat_to_label_id) * 10).softmax(dim=1)
        # ca1 = ca1.view(*ca1.shape[:2], -1).permute(0, 2, 1)
        # agg_map1 = get_agg_map(ca1, sa1, minmax_norm=False, ca_norm=False)
        # agg_map1 = upscale_attn(agg_map1, 320)
        # pred1 = agg_map1.argmax(dim=-1) + 1
        # pred1[bg_mask1.view_as(pred1)] = 0
        
        # multilabel classification
        cls_gt.append(val_gt.unique().tolist())
        cls_pred_0.append(pred0.unique().tolist())
        cls_pred_2.append(pred2.unique().tolist())

        for rs, pred in zip(rscores, [pred0, pred1, pred2, pred3, pred4, pred5, pred6]):
            rs.update(val_gt.cpu().numpy(), pred.cpu().numpy())

        mious = []
        for rs in rscores:
            metrics, _ = rs.get_scores()
            mious.append(metrics["Mean IoU"])

        description = " | ".join([f"mIoU_{i} {mious[i]:.3f}" for i in range(len(mious))])
        pbar.set_description(description)

    return rscores, (cls_gt, cls_pred_0, cls_pred_2)

with np.errstate(divide='ignore', invalid='ignore'):
    rs, cls_multilabel = run_gem_voc()

mIoU_0 0.566 | mIoU_1 0.607 | mIoU_2 0.558 | mIoU_3 0.611 | mIoU_4 0.699 | mIoU_5 0.567 | mIoU_6 0.572: 100%|██████████| 1464/1464 [1:19:03<00:00,  3.24s/it]


train: mIoU_0 0.566 | mIoU_1 0.607 | mIoU_2 0.558 | mIoU_3 0.611 | mIoU_4 0.699 | mIoU_5 0.567 | mIoU_6 0.572

val: mIoU_0 0.567 | mIoU_1 0.601 | mIoU_2 0.557 | mIoU_3 0.606 | mIoU_4 0.712 | mIoU_5 0.560 | mIoU_6 0.571

### train class IoUs
{'Pixel Acc': '81.2', 'Mean Acc': '83.7', 'FreqW Acc': '72.2', 'Mean IoU': '56.6'}
{'background': '78.1', 'aeroplane': '49.9', 'bicycle': '37.9', 'bird': '80.2', 'boat': '15.9', 'bottle': '60.4', 'bus': '73.1', 'car': '52.6', 'cat': '89.7', 'chair': '25.6', 'cow': '76.7', 'dining table': '37.3', 'dog': '81.5', 'horse': '72.0', 'motorbike': '67.2', 'person': '31.7', 'potted plant': '39.6', 'sheep': '80.6', 'sofa': '31.6', 'train': '63.3', 'tv/monitor': '43.9'}

####################################################################################################

{'Pixel Acc': '84.7', 'Mean Acc': '85.8', 'FreqW Acc': '75.8', 'Mean IoU': '60.7'}
{'background': '81.8', 'aeroplane': '49.8', 'bicycle': '38.2', 'bird': '80.0', 'boat': '20.8', 'bottle': '60.0', 'bus': '75.8', 'car': '62.0', 'cat': '86.9', 'chair': '44.0', 'cow': '78.3', 'dining table': '46.8', 'dog': '81.0', 'horse': '72.5', 'motorbike': '69.1', 'person': '31.3', 'potted plant': '55.4', 'sheep': '83.9', 'sofa': '49.0', 'train': '63.5', 'tv/monitor': '45.5'}

####################################################################################################

{'Pixel Acc': '81.4', 'Mean Acc': '82.1', 'FreqW Acc': '72.3', 'Mean IoU': '55.8'}
{'background': '78.0', 'aeroplane': '49.5', 'bicycle': '38.5', 'bird': '75.0', 'boat': '16.3', 'bottle': '56.1', 'bus': '67.9', 'car': '51.1', 'cat': '86.5', 'chair': '26.2', 'cow': '74.7', 'dining table': '36.9', 'dog': '79.5', 'horse': '70.7', 'motorbike': '70.3', 'person': '41.5', 'potted plant': '36.4', 'sheep': '78.5', 'sofa': '32.4', 'train': '64.0', 'tv/monitor': '42.6'}

####################################################################################################

{'Pixel Acc': '85.5', 'Mean Acc': '84.5', 'FreqW Acc': '76.7', 'Mean IoU': '61.1'}
{'background': '82.5', 'aeroplane': '49.0', 'bicycle': '39.0', 'bird': '70.6', 'boat': '22.0', 'bottle': '57.0', 'bus': '73.1', 'car': '65.2', 'cat': '84.6', 'chair': '45.7', 'cow': '78.8', 'dining table': '50.8', 'dog': '80.5', 'horse': '71.7', 'motorbike': '72.2', 'person': '40.1', 'potted plant': '57.3', 'sheep': '81.7', 'sofa': '52.2', 'train': '64.8', 'tv/monitor': '44.2'}

####################################################################################################

{'Pixel Acc': '86.5', 'Mean Acc': '89.7', 'FreqW Acc': '79.8', 'Mean IoU': '69.9'}
{'background': '83.9', 'aeroplane': '59.7', 'bicycle': '69.7', 'bird': '98.7', 'boat': '14.5', 'bottle': '74.2', 'bus': '87.9', 'car': '68.2', 'cat': '98.0', 'chair': '30.6', 'cow': '91.2', 'dining table': '49.1', 'dog': '93.5', 'horse': '89.5', 'motorbike': '86.1', 'person': '42.7', 'potted plant': '54.1', 'sheep': '91.3', 'sofa': '37.2', 'train': '81.8', 'tv/monitor': '66.3'}

####################################################################################################

{'Pixel Acc': '81.8', 'Mean Acc': '83.6', 'FreqW Acc': '72.8', 'Mean IoU': '56.7'}
{'background': '78.8', 'aeroplane': '55.5', 'bicycle': '34.3', 'bird': '74.7', 'boat': '16.3', 'bottle': '60.4', 'bus': '72.5', 'car': '53.0', 'cat': '86.4', 'chair': '24.3', 'cow': '75.9', 'dining table': '38.8', 'dog': '78.7', 'horse': '73.0', 'motorbike': '68.6', 'person': '36.7', 'potted plant': '40.4', 'sheep': '79.5', 'sofa': '32.1', 'train': '64.6', 'tv/monitor': '45.9'}

####################################################################################################

{'Pixel Acc': '81.8', 'Mean Acc': '83.6', 'FreqW Acc': '72.9', 'Mean IoU': '57.2'}
{'background': '78.7', 'aeroplane': '51.4', 'bicycle': '36.7', 'bird': '81.3', 'boat': '15.9', 'bottle': '61.8', 'bus': '71.4', 'car': '53.3', 'cat': '88.0', 'chair': '25.8', 'cow': '78.2', 'dining table': '37.8', 'dog': '81.6', 'horse': '72.0', 'motorbike': '67.9', 'person': '37.6', 'potted plant': '37.7', 'sheep': '82.9', 'sofa': '32.5', 'train': '65.1', 'tv/monitor': '43.5'}

####################################################################################################

### val class IoUs

{'Pixel Acc': '81.3', 'Mean Acc': '84.2', 'FreqW Acc': '72.2', 'Mean IoU': '56.7'}
{'background': '77.8', 'aeroplane': '48.1', 'bicycle': '33.6', 'bird': '82.6', 'boat': '14.2', 'bottle': '63.3', 'bus': '80.5', 'car': '53.9', 'cat': '87.4', 'chair': '28.8', 'cow': '76.8', 'dining table': '31.8', 'dog': '82.2', 'horse': '67.3', 'motorbike': '63.4', 'person': '35.2', 'potted plant': '34.2', 'sheep': '83.8', 'sofa': '34.6', 'train': '62.6', 'tv/monitor': '47.5'}

####################################################################################################

{'Pixel Acc': '84.6', 'Mean Acc': '85.8', 'FreqW Acc': '75.7', 'Mean IoU': '60.1'}
{'background': '81.6', 'aeroplane': '49.5', 'bicycle': '32.8', 'bird': '81.3', 'boat': '20.3', 'bottle': '60.8', 'bus': '82.9', 'car': '60.0', 'cat': '86.4', 'chair': '38.1', 'cow': '81.7', 'dining table': '42.4', 'dog': '80.6', 'horse': '70.3', 'motorbike': '63.8', 'person': '34.4', 'potted plant': '54.0', 'sheep': '83.8', 'sofa': '48.3', 'train': '62.8', 'tv/monitor': '47.4'}

####################################################################################################

{'Pixel Acc': '81.5', 'Mean Acc': '82.4', 'FreqW Acc': '72.4', 'Mean IoU': '55.7'}
{'background': '77.8', 'aeroplane': '47.4', 'bicycle': '36.6', 'bird': '74.9', 'boat': '14.5', 'bottle': '58.6', 'bus': '72.6', 'car': '54.6', 'cat': '83.7', 'chair': '29.7', 'cow': '76.0', 'dining table': '32.0', 'dog': '81.1', 'horse': '64.6', 'motorbike': '65.4', 'person': '44.7', 'potted plant': '32.8', 'sheep': '79.0', 'sofa': '35.8', 'train': '61.8', 'tv/monitor': '45.6'}

####################################################################################################

{'Pixel Acc': '85.5', 'Mean Acc': '84.3', 'FreqW Acc': '76.7', 'Mean IoU': '60.6'}
{'background': '82.4', 'aeroplane': '48.5', 'bicycle': '36.1', 'bird': '72.4', 'boat': '21.5', 'bottle': '57.8', 'bus': '77.9', 'car': '64.3', 'cat': '83.3', 'chair': '42.7', 'cow': '81.7', 'dining table': '46.2', 'dog': '80.7', 'horse': '67.7', 'motorbike': '67.0', 'person': '43.6', 'potted plant': '57.7', 'sheep': '79.5', 'sofa': '52.5', 'train': '63.1', 'tv/monitor': '45.4'}

####################################################################################################

{'Pixel Acc': '87.3', 'Mean Acc': '90.8', 'FreqW Acc': '80.9', 'Mean IoU': '71.2'}
{'background': '84.6', 'aeroplane': '62.3', 'bicycle': '54.8', 'bird': '98.6', 'boat': '14.6', 'bottle': '80.7', 'bus': '96.9', 'car': '74.8', 'cat': '97.8', 'chair': '43.7', 'cow': '93.8', 'dining table': '40.4', 'dog': '93.9', 'horse': '82.6', 'motorbike': '81.4', 'person': '46.2', 'potted plant': '44.1', 'sheep': '96.5', 'sofa': '38.7', 'train': '82.8', 'tv/monitor': '85.0'}

####################################################################################################

{'Pixel Acc': '81.4', 'Mean Acc': '83.9', 'FreqW Acc': '72.3', 'Mean IoU': '56.0'}
{'background': '78.0', 'aeroplane': '49.1', 'bicycle': '31.5', 'bird': '73.1', 'boat': '14.3', 'bottle': '64.7', 'bus': '76.9', 'car': '57.7', 'cat': '82.4', 'chair': '27.9', 'cow': '79.3', 'dining table': '33.2', 'dog': '78.3', 'horse': '67.3', 'motorbike': '64.6', 'person': '39.6', 'potted plant': '31.7', 'sheep': '82.0', 'sofa': '35.1', 'train': '63.0', 'tv/monitor': '46.3'}

####################################################################################################

{'Pixel Acc': '82.0', 'Mean Acc': '83.8', 'FreqW Acc': '73.1', 'Mean IoU': '57.1'}
{'background': '78.6', 'aeroplane': '50.8', 'bicycle': '33.5', 'bird': '82.0', 'boat': '14.5', 'bottle': '61.7', 'bus': '77.9', 'car': '56.9', 'cat': '85.1', 'chair': '28.9', 'cow': '81.2', 'dining table': '32.5', 'dog': '81.9', 'horse': '66.3', 'motorbike': '62.1', 'person': '40.7', 'potted plant': '34.5', 'sheep': '83.6', 'sofa': '36.3', 'train': '63.4', 'tv/monitor': '47.4'}

In [11]:
for i in range(len(rs)):
    metrics, cls_iou = rs[i].get_scores() # val results
    print({k: f"{v*100:.1f}" for k, v in metrics.items()})
    print({label_id_to_cat[i]: f"{v*100:.1f}" for i, v in cls_iou.items()})
    print("#" * 100)

{'Pixel Acc': '81.2', 'Mean Acc': '83.7', 'FreqW Acc': '72.2', 'Mean IoU': '56.6'}
{'background': '78.1', 'aeroplane': '49.9', 'bicycle': '37.9', 'bird': '80.2', 'boat': '15.9', 'bottle': '60.4', 'bus': '73.1', 'car': '52.6', 'cat': '89.7', 'chair': '25.6', 'cow': '76.7', 'dining table': '37.3', 'dog': '81.5', 'horse': '72.0', 'motorbike': '67.2', 'person': '31.7', 'potted plant': '39.6', 'sheep': '80.6', 'sofa': '31.6', 'train': '63.3', 'tv/monitor': '43.9'}
####################################################################################################
{'Pixel Acc': '84.7', 'Mean Acc': '85.8', 'FreqW Acc': '75.8', 'Mean IoU': '60.7'}
{'background': '81.8', 'aeroplane': '49.8', 'bicycle': '38.2', 'bird': '80.0', 'boat': '20.8', 'bottle': '60.0', 'bus': '75.8', 'car': '62.0', 'cat': '86.9', 'chair': '44.0', 'cow': '78.3', 'dining table': '46.8', 'dog': '81.0', 'horse': '72.5', 'motorbike': '69.1', 'person': '31.3', 'potted plant': '55.4', 'sheep': '83.9', 'sofa': '49.0', 'train': '

In [12]:
from sklearn.metrics import classification_report, precision_score, recall_score, multilabel_confusion_matrix, accuracy_score

def binarize_multilabel(labels, n_classes):
    # labels: list of list
    label_array = np.zeros((len(labels), n_classes))
    for i, label in enumerate(labels):
        label = [l for l in label if l != -1]
        label_array[i, label] = 1
    return label_array

cls_gt, cls_pred_0, cls_pred_2 = cls_multilabel
y_true = binarize_multilabel(cls_gt, len(label_id_to_cat))
y_pred0 = binarize_multilabel(cls_pred_0, len(label_id_to_cat))
y_pred2 = binarize_multilabel(cls_pred_2, len(label_id_to_cat))

pr0 = precision_score(y_true, y_pred0, average="macro")
rc0 = recall_score(y_true, y_pred0, average="macro")
pr2 = precision_score(y_true, y_pred2, average="macro")
rc2 = recall_score(y_true, y_pred2, average="macro")

print("Average precision 0 {:.3f}, average recall 0 {:.3f}".format(pr0, rc0))
print("Average precision 2 {:.3f}, average recall 2 {:.3f}".format(pr2, rc2))

Average precision 0 0.859, average recall 0 0.822
Average precision 2 0.783, average recall 2 0.906


### train results
Average precision 0 0.859, average recall 0 0.822

Average precision 2 0.783, average recall 2 0.906

In [13]:
print(classification_report(y_true, y_pred0))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1460
           1       1.00      0.98      0.99        88
           2       0.98      0.89      0.94        65
           3       0.99      0.86      0.92       105
           4       0.64      0.97      0.77        78
           5       0.88      0.48      0.62        87
           6       0.96      0.88      0.92        78
           7       0.69      0.84      0.76       128
           8       0.98      0.98      0.98       131
           9       0.68      0.36      0.47       148
          10       0.83      0.84      0.84        64
          11       0.55      0.96      0.70        82
          12       0.95      0.90      0.92       121
          13       0.97      0.93      0.95        68
          14       0.89      0.89      0.89        81
          15       0.97      0.35      0.51       442
          16       0.62      0.74      0.67        82
          17       0.98    

TRAIN

precision    recall  f1-score   support

           0       1.00      0.96      0.98      1460
           1       1.00      0.98      0.99        88
           2       0.98      0.89      0.94        65
           3       0.99      0.86      0.92       105
           4       0.64      0.97      0.77        78
           5       0.88      0.48      0.62        87
           6       0.96      0.88      0.92        78
           7       0.69      0.84      0.76       128
           8       0.98      0.98      0.98       131
           9       0.68      0.36      0.47       148
          10       0.83      0.84      0.84        64
          11       0.55      0.96      0.70        82
          12       0.95      0.90      0.92       121
          13       0.97      0.93      0.95        68
          14       0.89      0.89      0.89        81
          15       0.97      0.35      0.51       442
          16       0.62      0.74      0.67        82
          17       0.98      0.87      0.92        63
          18       0.57      0.90      0.70        93
          19       0.98      0.96      0.97        83
          20       0.95      0.71      0.81        83

   micro avg       0.90      0.82      0.86      3630
   macro avg       0.86      0.82      0.82      3630
weighted avg       0.92      0.82      0.84      3630
 samples avg       0.92      0.85      0.87      3630

In [14]:
print(classification_report(y_true, y_pred2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1460
           1       0.97      1.00      0.98        88
           2       0.92      0.94      0.93        65
           3       0.99      0.93      0.96       105
           4       0.52      0.99      0.68        78
           5       0.75      0.61      0.67        87
           6       0.86      0.95      0.90        78
           7       0.56      0.93      0.70       128
           8       0.94      0.99      0.97       131
           9       0.58      0.62      0.60       148
          10       0.63      0.92      0.75        64
          11       0.38      1.00      0.55        82
          12       0.85      0.95      0.89       121
          13       0.97      0.99      0.98        68
          14       0.80      0.96      0.88        81
          15       0.96      0.62      0.76       442
          16       0.46      0.85      0.60        82
          17       0.98    

TRAIN

precision    recall  f1-score   support

           0       1.00      1.00      1.00      1460
           1       0.97      1.00      0.98        88
           2       0.92      0.94      0.93        65
           3       0.99      0.93      0.96       105
           4       0.52      0.99      0.68        78
           5       0.75      0.61      0.67        87
           6       0.86      0.95      0.90        78
           7       0.56      0.93      0.70       128
           8       0.94      0.99      0.97       131
           9       0.58      0.62      0.60       148
          10       0.63      0.92      0.75        64
          11       0.38      1.00      0.55        82
          12       0.85      0.95      0.89       121
          13       0.97      0.99      0.98        68
          14       0.80      0.96      0.88        81
          15       0.96      0.62      0.76       442
          16       0.46      0.85      0.60        82
          17       0.98      0.92      0.95        63
          18       0.47      0.94      0.62        93
          19       0.92      1.00      0.96        83
          20       0.93      0.90      0.91        83

   micro avg       0.83      0.91      0.87      3630
   macro avg       0.78      0.91      0.82      3630
weighted avg       0.88      0.91      0.88      3630
 samples avg       0.86      0.93      0.88      3630

In [15]:
def avg_fpr(y_true, y_pred):
    conf_matrices = multilabel_confusion_matrix(y_true, y_pred)

    # Calculate False Positive Rate for each class
    false_positive_rates = []

    for i in range(len(conf_matrices)):
        false_positive_rate = conf_matrices[i][0, 1] / (conf_matrices[i][0, 1] + conf_matrices[i][0, 0])
        false_positive_rates.append(false_positive_rate)

    # Calculate the average false positive rate
    average_false_positive_rate = sum(false_positive_rates) / len(false_positive_rates)

    print("Average False Positive Rate:", average_false_positive_rate)

avg_fpr(y_true, y_pred0)
avg_fpr(y_true, y_pred2)

Average False Positive Rate: 0.023650317844157356
Average False Positive Rate: 0.07195985287287812


val results

Average False Positive Rate: 0.023650317844157356

Average False Positive Rate: 0.07195985287287812